In [1]:
# Animated train movement with speed labels and explicit direction control

import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Rectangle, Polygon, Circle, FancyBboxPatch
from matplotlib.patches import Wedge

# Define multiple scenarios
scenarios = {
    'demo': [
        ('direction', 'FORWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 10),
        ('direction', 'BACKWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 10)
    ],
    'scenario_1': [
        ('direction', 'FORWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 8),
        ('wait', 10),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 12),
        ('wait', 100),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 8),
        ('wait', 10),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 20),
        ('direction', 'BACKWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 8),
        ('wait', 10),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 12),
        ('wait', 100),
        ('set_speed', 10),
        ('wait', 10),
        ('set_speed', 8),
        ('wait', 10),
        ('set_speed', 6),
        ('wait', 10),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 20)
    ],
    'scenario_2': [
        ('direction', 'FORWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 10),
        ('wait', 100),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 30),
        ('direction', 'BACKWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 10),
        ('wait', 50),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 30),
        ('direction', 'FORWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 10),
        ('wait', 100),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 20),
        ('direction', 'BACKWARD'),
        ('status', 'POWER_ON'),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 10),
        ('wait', 150),
        ('set_speed', 6),
        ('wait', 30),
        ('set_speed', 0),
        ('status', 'POWER_OFF'),
        ('wait', 30)
    ]
}

def create_track_geometry(ax, track_length):
    """Draw track with details."""
    # Main track lines
    ax.plot([0, track_length], [0, 0], color='#555', linewidth=4, zorder=1)
    ax.plot([0, track_length], [-0.15, -0.15], color='#333', linewidth=2, zorder=1)

    # Track markers (sleepers)
    for m in range(0, int(track_length) + 1, 10):
        ax.plot([m, m], [-0.9, 0.9], color='#999', linewidth=1.5, zorder=0)


def create_train_geometry(ax, initial_x, train_width, train_height):
    """Create all train components and return them in a dictionary."""
    wheel_r = 0.45
    roof_y = train_height/2 - 0.15

    # Main body with rounded corners
    train = FancyBboxPatch((initial_x - train_width / 2, -train_height / 2), 
                          train_width, train_height,
                          boxstyle="round,pad=0.1", facecolor='#d32f2f', 
                          edgecolor='#b71c1c', linewidth=2, zorder=3)
    ax.add_patch(train)

    # Cabin section (darker red)
    cabin_width = train_width * 0.35
    cabin = Rectangle((initial_x - train_width/2, -train_height/2), 
                     cabin_width, train_height,
                     facecolor='#c62828', edgecolor='none', zorder=4)
    ax.add_patch(cabin)

    # Nose (aerodynamic wedge)
    nose_offsets = [
        (train_width/2, -train_height/2),
        (train_width/2 + 2.2, 0.0),
        (train_width/2, train_height/2)
    ]
    nose = Polygon([[initial_x + dx, dy] for dx, dy in nose_offsets], closed=True,
                   facecolor='#b71c1c', edgecolor='#8e0000', 
                   linewidth=1.8, zorder=5)
    ax.add_patch(nose)

    # Roof section
    roof = Rectangle((initial_x - train_width/2 + 0.2, roof_y), 
                    train_width - 0.4, 0.3,
                    facecolor='#9e9e9e', edgecolor='#757575', 
                    linewidth=1, zorder=6)
    ax.add_patch(roof)

    # Pantograph (on roof)
    panto_x = initial_x - train_width/2 + train_width * 0.3
    pantograph = ax.plot([panto_x, panto_x], [roof_y + 0.3, roof_y + 1.2], 
                        color='#424242', linewidth=2.5, zorder=7)[0]
    panto_arm = ax.plot([panto_x - 0.6, panto_x + 0.6], 
                       [roof_y + 1.2, roof_y + 1.2],
                       color='#424242', linewidth=2, zorder=7)[0]

    # Speed stripe (color-coded)
    stripe_height = 0.6
    stripe = Rectangle((initial_x - train_width/2 + 0.5, 0.4), 
                      train_width - 1.0, stripe_height,
                      facecolor='#2e7d32', edgecolor='none', zorder=4)
    ax.add_patch(stripe)

    # Windows
    window_w, window_h = 1.4, 0.8
    window_offsets = [1.2, 3.2, 5.2, 7.2, 9.2, 11.2, 13.2]
    windows = [
        Rectangle((initial_x - train_width/2 + off, 0.8), window_w, window_h,
                 facecolor='#4fc3f7', edgecolor='#0288d1', 
                 linewidth=1.2, zorder=5)
        for off in window_offsets
    ]
    for w in windows:
        ax.add_patch(w)

    # Door indicator
    door_x = initial_x - train_width/2 + train_width * 0.65
    door = Rectangle((door_x, -train_height/2 + 0.1), 1.8, train_height - 0.2,
                    facecolor='#fdd835', edgecolor='#f9a825', 
                    linewidth=1.5, zorder=5)
    ax.add_patch(door)

    # Wheels with details
    wheel_offsets = [-train_width*0.35, -train_width*0.15, 
                    train_width*0.15, train_width*0.35]
    wheels = []
    wheel_centers = []
    for off in wheel_offsets:
        # Main wheel
        wheel = Circle((initial_x + off, -1.8), radius=wheel_r, 
                      facecolor='#37474f', edgecolor='#263238', 
                      linewidth=2, zorder=4)
        ax.add_patch(wheel)
        wheels.append(wheel)
        # Inner hub
        hub = Circle((initial_x + off, -1.8), radius=wheel_r*0.4, 
                    facecolor='#78909c', edgecolor='#546e7a', 
                    linewidth=1, zorder=5)
        ax.add_patch(hub)
        wheel_centers.append(hub)

    # Undercarriage
    undercarriage = Rectangle((initial_x - train_width/2 + 1, -train_height/2 - 0.4), 
                             train_width - 2, 0.35,
                             facecolor='#455a64', edgecolor='#37474f', 
                             linewidth=1, zorder=2)
    ax.add_patch(undercarriage)

    # Headlights (dual)
    headlight_top = Circle((initial_x + train_width/2 + 1.5, 0.5), 
                          radius=0.28, facecolor='#fff9c4', 
                          edgecolor='#fbc02d', linewidth=1.5, zorder=6)
    headlight_bottom = Circle((initial_x + train_width/2 + 1.5, -0.5), 
                             radius=0.28, facecolor='#fff9c4', 
                             edgecolor='#fbc02d', linewidth=1.5, zorder=6)
    ax.add_patch(headlight_top)
    ax.add_patch(headlight_bottom)

    # Front bumper
    bumper = Rectangle((initial_x + train_width/2 + 2.2, -0.3), 0.15, 0.6,
                      facecolor='#424242', edgecolor='#212121', 
                      linewidth=1, zorder=5)
    ax.add_patch(bumper)

    # Return all components in a dictionary for easy access
    return {
        'train': train,
        'cabin': cabin,
        'nose': nose,
        'nose_offsets': nose_offsets,
        'roof': roof,
        'roof_y': roof_y,
        'pantograph': pantograph,
        'panto_arm': panto_arm,
        'stripe': stripe,
        'windows': windows,
        'window_offsets': window_offsets,
        'door': door,
        'wheels': wheels,
        'wheel_centers': wheel_centers,
        'wheel_offsets': wheel_offsets,
        'undercarriage': undercarriage,
        'headlight_top': headlight_top,
        'headlight_bottom': headlight_bottom,
        'bumper': bumper
    }


def generate_animation(scenario_events, output_filename):
    """Generate an animated GIF for a given scenario."""

    # Build per-second series: time, speed (km/h), position (axis units)
    track_length = 200.0
    scale_units_per_mps = 0.3  # tune to control how far the train moves for a given speed

    time = [0]
    speed_kmh = [0.0]
    pos = [15.0]  # start near the left

    direction = 1  # +1: FORWARD (right), -1: BACKWARD (left)
    current_speed = 0.0  # current speed in km/h

    t = 0
    x = pos[0]

    for event in scenario_events:
        if event[0] == 'status':
            # Handle POWER_ON / POWER_OFF - currently just continue
            pass
        elif event[0] == 'direction':
            # Set direction: 'FORWARD' = +1 (right), 'BACKWARD' = -1 (left)
            direction = +1 if event[1] == 'FORWARD' else -1
        elif event[0] == 'set_speed':
            # Set the current speed
            current_speed = float(event[1])
        elif event[0] == 'wait':
            # Wait for specified duration at current speed
            duration_s = event[1]
            v_mps = current_speed * 1000.0 / 3600.0
            for _ in range(int(duration_s)):
                t += 1
                time.append(t)
                speed_kmh.append(current_speed)

                # Move according to current speed and direction
                x += direction * v_mps * scale_units_per_mps

                # Keep within track bounds
                if x < 0:
                    x = 0
                elif x > track_length:
                    x = track_length

                pos.append(x)

    # Set up animation figure
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.set_xlim(0, track_length)
    ax.set_ylim(-6, 6)
    ax.axis('off')

    # Create track geometry
    create_track_geometry(ax, track_length)

    # Create train geometry
    train_width = 25.0
    train_height = 3.0
    train_parts = create_train_geometry(ax, pos[0], train_width, train_height)

    # Speed and direction label near the train
    label = ax.text(pos[0], 3.8, '', ha='center', va='bottom', fontsize=12, 
                   fontweight='bold', color='#1a237e')
    # Global title/info (time only)
    info = ax.text(track_length / 2, 5.2, '', ha='center', va='bottom', 
                  fontsize=13, fontweight='bold', color='#111')

    # Helper to get direction arrow for each frame by comparing positions
    def direction_arrow(i):
        if i == 0:
            return '→'
        dx = pos[i] - pos[i - 1]
        if dx > 1e-6:
            return '→'
        elif dx < -1e-6:
            return '←'
        else:
            return '•'

    # Helper to update train x-position
    def set_train_x(x):
        # Body
        train_parts['train'].set_x(x - train_width / 2)
        # Cabin
        train_parts['cabin'].set_x(x - train_width/2)
        # Roof
        train_parts['roof'].set_x(x - train_width/2 + 0.2)
        # Stripe
        train_parts['stripe'].set_x(x - train_width/2 + 0.5)
        # Windows
        for patch, off in zip(train_parts['windows'], train_parts['window_offsets']):
            patch.set_x(x - train_width/2 + off)
        # Door
        train_parts['door'].set_x(x - train_width/2 + train_width * 0.65)
        # Wheels and hubs
        for wheel, hub, off in zip(train_parts['wheels'], 
                                   train_parts['wheel_centers'], 
                                   train_parts['wheel_offsets']):
            wheel.center = (x + off, -1.8)
            hub.center = (x + off, -1.8)
        # Nose
        train_parts['nose'].set_xy([[x + dx, dy] for dx, dy in train_parts['nose_offsets']])
        # Headlights
        train_parts['headlight_top'].center = (x + train_width/2 + 1.5, 0.5)
        train_parts['headlight_bottom'].center = (x + train_width/2 + 1.5, -0.5)
        # Bumper
        train_parts['bumper'].set_x(x + train_width/2 + 2.2)
        # Undercarriage
        train_parts['undercarriage'].set_x(x - train_width/2 + 1)
        # Pantograph
        roof_y = train_parts['roof_y']
        panto_x = x - train_width/2 + train_width * 0.3
        train_parts['pantograph'].set_data([panto_x, panto_x], [roof_y + 0.3, roof_y + 1.2])
        train_parts['panto_arm'].set_data([panto_x - 0.6, panto_x + 0.6], 
                                         [roof_y + 1.2, roof_y + 1.2])

    # Update function
    def update(frame):
        x = pos[frame]
        v = speed_kmh[frame]
        arr = direction_arrow(frame)

        set_train_x(x)
        label.set_position((x, 3.8))
        label.set_text(f"{arr}  {v:.0f} km/h")

        # Headlight brightness based on speed
        if v > 0:
            train_parts['headlight_top'].set_facecolor('#ffeb3b')
            train_parts['headlight_bottom'].set_facecolor('#ffeb3b')
        else:
            train_parts['headlight_top'].set_facecolor('#fff9c4')
            train_parts['headlight_bottom'].set_facecolor('#fff9c4')

        # Time only (position removed)
        info.set_text(f"Time: {time[frame]} s")

        return (train_parts['train'], train_parts['cabin'], train_parts['nose'], 
                train_parts['roof'], train_parts['stripe'], train_parts['door'], 
                train_parts['undercarriage'], train_parts['bumper'], 
                train_parts['pantograph'], train_parts['panto_arm'], 
                *train_parts['windows'], *train_parts['wheels'], 
                *train_parts['wheel_centers'], 
                train_parts['headlight_top'], train_parts['headlight_bottom'], 
                label, info)

    # Animate
    ani = FuncAnimation(fig, update, frames=len(time), interval=20, blit=False)

    # Save GIF
    writer = PillowWriter(fps=25)
    ani.save(output_filename, writer=writer, dpi=150)

    print(f"Generated: {output_filename}")

    plt.close(fig)

# Generate animations for all scenarios
for scenario_name, scenario_events in scenarios.items():
    output_path = f"./train_movement_{scenario_name}.gif"
    generate_animation(scenario_events, output_path)

print("\nAll animations generated successfully!")


Generated: ./train_movement_demo.gif
Generated: ./train_movement_scenario_1.gif
Generated: ./train_movement_scenario_2.gif

All animations generated successfully!
